In [1]:
# import libraries
import pandas as pd

### Gather data

In [2]:
# Download delays data manually choosing All major airports options
# data source:
# https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp?pn=1

# read the downloaded file 
df_read = pd.read_csv('381568907_72018_330_airline_delay_causes.csv')

In [3]:
# make a copy of the data
df = df_read.copy()

In [4]:
# Download geogrpahical data on airports
# data source: 
# https://opendata.socrata.com/dataset/AirportLocations/ia9d-4ymt
df_map = pd.read_csv('Airport_Codes_mapped_to_Latitude_Longitude_in_the_United_States.csv')

### Access

In [5]:
# quick look at the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63816 entries, 0 to 63815
Data columns (total 22 columns):
year                   63816 non-null int64
 month                 63816 non-null int64
carrier                63816 non-null object
carrier_name           63816 non-null object
airport                63816 non-null object
airport_name           63816 non-null object
arr_flights            63780 non-null float64
arr_del15              63777 non-null float64
carrier_ct             63780 non-null float64
 weather_ct            63780 non-null float64
nas_ct                 63780 non-null float64
security_ct            63780 non-null float64
late_aircraft_ct       63780 non-null float64
arr_cancelled          63780 non-null float64
arr_diverted           63780 non-null float64
 arr_delay             63780 non-null float64
 carrier_delay         63780 non-null float64
weather_delay          63780 non-null float64
nas_delay              63780 non-null float64
security_delay         63

##### Variables definition

Date:

- year
- month

Airport and airline information:
- carrier: airline code.
- carrier_name: airline name.
- airport: airport code.
- airport_name: airport name.

Flight and delays count:

- arr_flights: total number of flights which arrive at the airport (sum of flights arrived on time, delayed, cancelled and diverted)
- arr_del15: total number of flights delayed (over 15 minutes)
- carrier_ct: total umber of flights delayed due to carrier 
- weather_ct: total number of flights delayed due to weather
- nas_ct: total number of flights delayed due to National Aviation System
- security_ct: total number of flights delayed due to security
- late_aircraft_ct: total number of flights delayed due to previous flight with same aircraft arriving late and causing the present flight late departure
- arr_cancelled: total number of cancelled flights
- arr_diverted: total number of diverted flights

(Those are sometimes floats since one flight delay may be attributed to multiple causes and therefore distributed accross the causes accordingly.)

Delays length in minutes (total time):

- arr_delay 
- carrier_delay
- weather_delay
- nas_delay
- security_delay
- late_aircraft_delay


In [6]:
df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
0,2003,6,AA,American Airlines Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",752.0,186.0,33.99,27.82,...,17.53,5.0,0.0,8314.0,1367.0,1722.0,3817.0,139.0,1269.0,NaN
1,2003,6,AA,American Airlines Inc.,BOS,"Boston, MA: Logan International",1266.0,225.0,69.43,23.66,...,45.73,7.0,0.0,12139.0,4201.0,1783.0,3067.0,45.0,3043.0,NaN
2,2003,6,AA,American Airlines Inc.,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",593.0,101.0,17.56,20.49,...,24.69,1.0,1.0,5698.0,1058.0,1332.0,1708.0,0.0,1600.0,NaN
3,2003,6,AA,American Airlines Inc.,CLT,"Charlotte, NC: Charlotte Douglas International",203.0,48.0,16.57,9.72,...,9.76,4.0,0.0,3048.0,968.0,742.0,558.0,127.0,653.0,NaN
4,2003,6,AA,American Airlines Inc.,DCA,"Washington, DC: Ronald Reagan Washington National",947.0,147.0,42.88,24.89,...,26.44,5.0,2.0,7868.0,2048.0,1484.0,2767.0,3.0,1566.0,NaN


In [7]:
df.airport.unique()

array(['ATL', 'BOS', 'BWI', 'CLT', 'DCA', 'DEN', 'DFW', 'DTW', 'EWR',
       'FLL', 'HNL', 'IAD', 'IAH', 'JFK', 'LAS', 'LAX', 'LGA', 'MCO',
       'MDW', 'MIA', 'MSP', 'ORD', 'PDX', 'PHL', 'PHX', 'SAN', 'SEA',
       'SFO', 'SLC', 'TPA'], dtype=object)

In [8]:
df.carrier_name.value_counts()

Delta Air Lines Inc.            5422
American Airlines Inc.          5318
United Air Lines Inc.           5211
US Airways Inc.                 4032
ExpressJet Airlines Inc.        3624
Southwest Airlines Co.          3620
JetBlue Airways                 3597
Alaska Airlines Inc.            3558
Frontier Airlines Inc.          3531
SkyWest Airlines Inc.           3340
Continental Air Lines Inc.      2918
AirTran Airways Corporation     2801
American Eagle Airlines Inc.    2379
Northwest Airlines Inc.         2367
Mesa Airlines Inc.              1869
Comair Inc.                     1671
Hawaiian Airlines Inc.          1490
Atlantic Southeast Airlines     1460
Virgin America                  1192
Spirit Air Lines                 868
Pinnacle Airlines Inc.           823
America West Airlines Inc.       807
ATA Airlines d/b/a ATA           649
Envoy Air                        361
Endeavor Air Inc.                225
Independence Air                 186
Atlantic Coast Airlines          177
R

In [9]:
# check if the minimum delay length in minutes is truly not less than 15 for the delayed flights  
(df[' arr_delay'] / df['arr_del15']).describe()

count    63161.000000
mean        53.887255
std         18.193676
min         15.000000
25%         43.400000
50%         52.080292
75%         61.895238
max       1399.000000
dtype: float64

In [10]:
# calculate sum of minutes for different causes 
df[[' carrier_delay', 'weather_delay', 'nas_delay', 
                                  'security_delay', 'late_aircraft_delay']].sum(axis=1, min_count=5).describe()

count     63780.000000
mean      11352.223111
std       23279.588686
min           0.000000
25%        1298.000000
50%        3655.000000
75%        9818.250000
max      433687.000000
dtype: float64

In [11]:
# check if the sum values are the same as values in the arr_delay column
df[' arr_delay'].describe()

count     63780.000000
mean      11352.223111
std       23279.588686
min           0.000000
25%        1298.000000
50%        3655.000000
75%        9818.250000
max      433687.000000
Name:  arr_delay, dtype: float64

In [12]:
# calculate sum of counts for different causes 
df[['carrier_ct', ' weather_ct', 'nas_ct', 'security_ct', 
                   'late_aircraft_ct']].sum(axis=1, min_count=5).describe()

count    63780.000000
mean       195.873164
std        371.633496
min          0.000000
25%         26.000000
50%         70.000000
75%        175.000000
max       6377.000000
dtype: float64

In [13]:
# check if the sum values are the same as values in the arr_delay_min column
df['arr_del15'].describe()

count    63777.000000
mean       195.882309
std        371.639789
min          0.000000
25%         26.000000
50%         70.000000
75%        175.000000
max       6377.000000
Name: arr_del15, dtype: float64

In [14]:
df[pd.isna(df.arr_flights)]

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
1291,2003,9,RU,ExpressJet Airlines Inc.,TPA,"Tampa, FL: Tampa International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3151,2004,2,TZ,ATA Airlines d/b/a ATA,TPA,"Tampa, FL: Tampa International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3522,2004,3,TZ,ATA Airlines d/b/a ATA,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3745,2004,4,EV,Atlantic Southeast Airlines,LAX,"Los Angeles, CA: Los Angeles International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11883,2005,12,TZ,ATA Airlines d/b/a ATA,MSP,"Minneapolis, MN: Minneapolis-St Paul Internati...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11884,2005,12,TZ,ATA Airlines d/b/a ATA,ORD,"Chicago, IL: Chicago O'Hare International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12249,2006,1,TZ,ATA Airlines d/b/a ATA,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18562,2007,5,EV,Atlantic Southeast Airlines,ORD,"Chicago, IL: Chicago O'Hare International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18675,2007,5,OH,Comair Inc.,FLL,"Fort Lauderdale, FL: Fort Lauderdale-Hollywood...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19490,2007,7,OH,Comair Inc.,DEN,"Denver, CO: Denver International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df[pd.isna(df.arr_del15)]

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
1291,2003,9,RU,ExpressJet Airlines Inc.,TPA,"Tampa, FL: Tampa International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3151,2004,2,TZ,ATA Airlines d/b/a ATA,TPA,"Tampa, FL: Tampa International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3522,2004,3,TZ,ATA Airlines d/b/a ATA,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3745,2004,4,EV,Atlantic Southeast Airlines,LAX,"Los Angeles, CA: Los Angeles International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11883,2005,12,TZ,ATA Airlines d/b/a ATA,MSP,"Minneapolis, MN: Minneapolis-St Paul Internati...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11884,2005,12,TZ,ATA Airlines d/b/a ATA,ORD,"Chicago, IL: Chicago O'Hare International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12249,2006,1,TZ,ATA Airlines d/b/a ATA,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18562,2007,5,EV,Atlantic Southeast Airlines,ORD,"Chicago, IL: Chicago O'Hare International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18675,2007,5,OH,Comair Inc.,FLL,"Fort Lauderdale, FL: Fort Lauderdale-Hollywood...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19490,2007,7,OH,Comair Inc.,DEN,"Denver, CO: Denver International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Clean

#### Format column names properly

Get read of spaces in column names

In [16]:
# strip column names in order to access them easier
df.columns = df.columns.str.strip()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63816 entries, 0 to 63815
Data columns (total 22 columns):
year                   63816 non-null int64
month                  63816 non-null int64
carrier                63816 non-null object
carrier_name           63816 non-null object
airport                63816 non-null object
airport_name           63816 non-null object
arr_flights            63780 non-null float64
arr_del15              63777 non-null float64
carrier_ct             63780 non-null float64
weather_ct             63780 non-null float64
nas_ct                 63780 non-null float64
security_ct            63780 non-null float64
late_aircraft_ct       63780 non-null float64
arr_cancelled          63780 non-null float64
arr_diverted           63780 non-null float64
arr_delay              63780 non-null float64
carrier_delay          63780 non-null float64
weather_delay          63780 non-null float64
nas_delay              63780 non-null float64
security_delay         63

#### Join flights/delays data and geographical data for airports

Join both DataFrames, but before filter the df_map to only major airports from the df

In [17]:
# which airports are included into df 
airports = df.airport.unique()
airports

array(['ATL', 'BOS', 'BWI', 'CLT', 'DCA', 'DEN', 'DFW', 'DTW', 'EWR',
       'FLL', 'HNL', 'IAD', 'IAH', 'JFK', 'LAS', 'LAX', 'LGA', 'MCO',
       'MDW', 'MIA', 'MSP', 'ORD', 'PDX', 'PHL', 'PHX', 'SAN', 'SEA',
       'SFO', 'SLC', 'TPA'], dtype=object)

In [18]:
# filter to reduce only to airports which are in the flights df 
df_map = df_map[df_map.locationID.isin(airports)].reset_index(drop=True)

In [19]:
# rename df_map column with airport IDs the same a the column in df
df_map = df_map.rename(columns={'locationID': 'airport'})

In [20]:
# join df_map and df by airport column
df = df.join(df_map.set_index('airport'), on='airport')

In [21]:
df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21,Latitude,Longitude
0,2003,6,AA,American Airlines Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",752.0,186.0,33.99,27.82,...,0.0,8314.0,1367.0,1722.0,3817.0,139.0,1269.0,NaN,33.6367,84.4281
1,2003,6,AA,American Airlines Inc.,BOS,"Boston, MA: Logan International",1266.0,225.0,69.43,23.66,...,0.0,12139.0,4201.0,1783.0,3067.0,45.0,3043.0,NaN,42.3631,71.0064
2,2003,6,AA,American Airlines Inc.,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",593.0,101.0,17.56,20.49,...,1.0,5698.0,1058.0,1332.0,1708.0,0.0,1600.0,NaN,39.1753,76.6683
3,2003,6,AA,American Airlines Inc.,CLT,"Charlotte, NC: Charlotte Douglas International",203.0,48.0,16.57,9.72,...,0.0,3048.0,968.0,742.0,558.0,127.0,653.0,NaN,35.2139,80.9431
4,2003,6,AA,American Airlines Inc.,DCA,"Washington, DC: Ronald Reagan Washington National",947.0,147.0,42.88,24.89,...,2.0,7868.0,2048.0,1484.0,2767.0,3.0,1566.0,NaN,38.8522,77.0378


In [22]:
df.columns.tolist()

['year',
 'month',
 'carrier',
 'carrier_name',
 'airport',
 'airport_name',
 'arr_flights',
 'arr_del15',
 'carrier_ct',
 'weather_ct',
 'nas_ct',
 'security_ct',
 'late_aircraft_ct',
 'arr_cancelled',
 'arr_diverted',
 'arr_delay',
 'carrier_delay',
 'weather_delay',
 'nas_delay',
 'security_delay',
 'late_aircraft_delay',
 'Unnamed: 21',
 'Latitude',
 'Longitude']

In [23]:
#change column locations for Latitude and Longitude

col_titles = ['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name', 'Latitude', 'Longitude',
              'arr_flights', 'arr_del15','carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct', 
              'arr_cancelled', 'arr_diverted',
              'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']
df = df.reindex(columns=col_titles)

#### Create a date column from month and year

Combine month and year in one column and change type to datetime

In [24]:
# join year and month columns and change the datatype to datetime
df['date'] = df.year.astype('str') + '-' + df.month.astype('str')
df.date = pd.to_datetime(df.date, format='%Y-%m')

#change column location for date
col_titles = [df.columns.tolist()[-1]] + [item for item in df.columns.tolist()[0:-1]] 
df = df.reindex(columns=col_titles)

In [25]:
# drop columns which are no longer neeed
df = df.drop(['year', 'month'], axis=1)

In [26]:
df.date.describe()

count                   63816
unique                    182
top       2007-06-01 00:00:00
freq                      416
first     2003-06-01 00:00:00
last      2018-07-01 00:00:00
Name: date, dtype: object

#### Eliminate entries with no data on flights

Delete rows where no information on flight count and delays is stored

In [27]:
# delete rows where no informatino is available for arrived flights
df = df.drop(index = df.index[pd.isna(df.arr_flights)])

In [28]:
# delete rows where no informatino is available for delayed flights
df = df.drop(index = df.index[pd.isna(df.arr_del15)])

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63777 entries, 0 to 63815
Data columns (total 22 columns):
date                   63777 non-null datetime64[ns]
carrier                63777 non-null object
carrier_name           63777 non-null object
airport                63777 non-null object
airport_name           63777 non-null object
Latitude               63777 non-null float64
Longitude              63777 non-null float64
arr_flights            63777 non-null float64
arr_del15              63777 non-null float64
carrier_ct             63777 non-null float64
weather_ct             63777 non-null float64
nas_ct                 63777 non-null float64
security_ct            63777 non-null float64
late_aircraft_ct       63777 non-null float64
arr_cancelled          63777 non-null float64
arr_diverted           63777 non-null float64
arr_delay              63777 non-null float64
carrier_delay          63777 non-null float64
weather_delay          63777 non-null float64
nas_delay     

In [30]:
df = df.reset_index(drop=True)

In [31]:
df.tail()

,date,carrier,carrier_name,airport,airport_name,Latitude,Longitude,arr_flights,arr_del15,carrier_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
63772,2018-07-01,YX,Republic Airline,MSP,"Minneapolis, MN: Minneapolis-St Paul Internati...",44.8819,93.2217,455.0,90.0,22.09,...,0.0,27.57,17.0,1.0,5418.0,1176.0,298.0,1884.0,0.0,2060.0
63773,2018-07-01,YX,Republic Airline,ORD,"Chicago, IL: Chicago O'Hare International",41.9808,87.9067,1572.0,261.0,30.73,...,0.0,100.18,36.0,1.0,19677.0,1975.0,1314.0,7447.0,0.0,8941.0
63774,2018-07-01,YX,Republic Airline,PHL,"Philadelphia, PA: Philadelphia International",39.8722,75.2408,1691.0,339.0,38.98,...,0.0,125.80,104.0,0.0,20604.0,2162.0,654.0,9724.0,0.0,8064.0
63775,2018-07-01,YX,Republic Airline,SLC,"Salt Lake City, UT: Salt Lake City International",40.7883,111.9778,59.0,13.0,4.61,...,0.0,4.64,1.0,0.0,1196.0,410.0,176.0,114.0,0.0,496.0
63776,2018-07-01,YX,Republic Airline,TPA,"Tampa, FL: Tampa International",27.9756,82.5333,27.0,4.0,0.00,...,0.0,0.00,0.0,0.0,112.0,0.0,0.0,112.0,0.0,0.0


#### Create a new variable for flights on time count

Since all flights consist of those which are on time, just delayed (over 15 minutes), cancelled and diverted, calculate count for flights on time from other variables which are known

In [32]:
# create a new column and calculate count of flights which are on time
df['arr_ontime'] = df['arr_flights'] - df[['arr_del15', 'arr_cancelled', 'arr_diverted']].sum(axis=1)

In [33]:
df['arr_ontime'].sample(10)

31268    2723.0
5388      275.0
47573    2907.0
9764       30.0
40143      92.0
23694     109.0
18063     655.0
41636      76.0
33361    2348.0
8224     2971.0
Name: arr_ontime, dtype: float64

In [34]:
df.columns.tolist()

['date',
 'carrier',
 'carrier_name',
 'airport',
 'airport_name',
 'Latitude',
 'Longitude',
 'arr_flights',
 'arr_del15',
 'carrier_ct',
 'weather_ct',
 'nas_ct',
 'security_ct',
 'late_aircraft_ct',
 'arr_cancelled',
 'arr_diverted',
 'arr_delay',
 'carrier_delay',
 'weather_delay',
 'nas_delay',
 'security_delay',
 'late_aircraft_delay',
 'arr_ontime']

In [35]:
# reorder columns
df = df[['date', 'carrier', 'carrier_name', 'airport', 'airport_name', 'Latitude', 'Longitude',
 'arr_flights','arr_ontime', 'arr_del15','arr_cancelled', 'arr_diverted',
 'carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct',
 'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']]

#### Create two separate DataFrames: one with categories of flights and second with categories of delay causes and corresponding delay length in minutes

Flights can be categorized in those which are on time, delayed, cancelled or diverted. There is count information provided on them.

Delays on the other hand comprise flights which are delayed (one of the categories from mentioned above) and consist of categories of delays with different causes: carrier, nas, weather, security and late aircarft. There is count and lenght in minutes available for them.

*dimensions:* 
 'date',
 'carrier',
 'carrier_name',
 'airport',
 'airport_name',
 'Latitude',
 'Longitude'

*counts 1st category:*
 'arr_del15'
 'arr_cancelled',
 'arr_diverted',
 'arr_ontime'

*counts 2st category:*
 'carrier_ct',
 'weather_ct',
 'nas_ct',
 'security_ct',
 'late_aircraft_ct'
 
*length in minutes 2st category:*
 'carrier_delay',
 'weather_delay',
 'nas_delay',
 'security_delay',
 'late_aircraft_delay'
 
*sums of counts for 1st and 2nd level categories (to drop):* 
 'arr_flights',
 'arr_delay'

In [36]:
# melt arr_ontime, arr_del15, arr_cancelled, arr_diverted into one column to create a separate category, create first df
df_flights = pd.melt(df, 
              id_vars=['date', 'carrier', 'carrier_name', 'airport', 'airport_name', 'Latitude', 'Longitude'],
              value_vars=['arr_del15', 'arr_cancelled', 'arr_diverted', 'arr_ontime'],
              var_name='arr_category',
              value_name='arr_count')

In [37]:
df_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255108 entries, 0 to 255107
Data columns (total 9 columns):
date            255108 non-null datetime64[ns]
carrier         255108 non-null object
carrier_name    255108 non-null object
airport         255108 non-null object
airport_name    255108 non-null object
Latitude        255108 non-null float64
Longitude       255108 non-null float64
arr_category    255108 non-null object
arr_count       255108 non-null float64
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 17.5+ MB


In [38]:
df_flights.head(10)

,date,carrier,carrier_name,airport,airport_name,Latitude,Longitude,arr_category,arr_count
0,2003-06-01,AA,American Airlines Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",33.6367,84.4281,arr_del15,186.0
1,2003-06-01,AA,American Airlines Inc.,BOS,"Boston, MA: Logan International",42.3631,71.0064,arr_del15,225.0
2,2003-06-01,AA,American Airlines Inc.,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",39.1753,76.6683,arr_del15,101.0
3,2003-06-01,AA,American Airlines Inc.,CLT,"Charlotte, NC: Charlotte Douglas International",35.2139,80.9431,arr_del15,48.0
4,2003-06-01,AA,American Airlines Inc.,DCA,"Washington, DC: Ronald Reagan Washington National",38.8522,77.0378,arr_del15,147.0
5,2003-06-01,AA,American Airlines Inc.,DEN,"Denver, CO: Denver International",39.8617,104.6731,arr_del15,157.0
6,2003-06-01,AA,American Airlines Inc.,DFW,"Dallas/Fort Worth, TX: Dallas/Fort Worth Inter...",32.8969,97.0381,arr_del15,2144.0
7,2003-06-01,AA,American Airlines Inc.,DTW,"Detroit, MI: Detroit Metro Wayne County",42.2125,83.3533,arr_del15,82.0
8,2003-06-01,AA,American Airlines Inc.,EWR,"Newark, NJ: Newark Liberty International",40.6925,74.1686,arr_del15,149.0
9,2003-06-01,AA,American Airlines Inc.,FLL,"Fort Lauderdale, FL: Fort Lauderdale-Hollywood...",26.0726,80.1528,arr_del15,146.0


In [39]:
# melt 'arr_ontime', 'carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct' into one column, create first df
df_delays_1 = pd.melt(df, 
                id_vars=['date', 'carrier', 'carrier_name', 'airport', 'airport_name', 'Latitude', 'Longitude'],
                value_vars=['carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct'],
                var_name='delay_cause',
                value_name='delay_ct')

In [40]:
# melt carrier_delay_min', 'weather_delay_min', 'nas_delay_min', 'security_delay_min','late_aircraft_delay_min' into 
# another column, create second df

df_delays_2 = pd.melt(df, 
               id_vars=['date', 'carrier', 'airport'],
              value_vars=['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay'],
              var_name='delay_cause',
              value_name='delay_min')

In [41]:
# strip _ct and _delay parts of the variable values in the delay_cause column
df_delays_1.delay_cause = df_delays_1.delay_cause.str[:-len('_ct')]
df_delays_2.delay_cause = df_delays_2.delay_cause.str[:-len('_delay')]

In [42]:
# concat two Dataframes by setting index to four same columns 
# https://stackoverflow.com/questions/51722864/reshaping-pandas-dataframe-python-3-x?noredirect=1&lq=1

In [43]:
df_delays_1 = df_delays_1.set_index(['date', 'carrier', 'airport', 'delay_cause', 
                                     df_delays_1.groupby(['date', 'carrier', 'airport', 'delay_cause']).cumcount()
                                    ])

In [44]:
df_delays_2 = df_delays_2.set_index(['date', 'carrier', 'airport', 'delay_cause', 
                     df_delays_2.groupby(['date', 'carrier', 'airport', 'delay_cause']).cumcount()
                    ])

In [45]:
df_delays = pd.concat([df_delays_1, df_delays_2], axis=1
                     ).sort_index(level=4).reset_index(level=4, drop=True).reset_index()


In [46]:
# create a arr_category in the df_delays column and set value to delay 
# in order to be able to join both delays and flights DataFrames if needed later

df_delays['arr_category'] = 'arr_del15'

In [47]:
df_delays.columns.tolist()

['date',
 'carrier',
 'airport',
 'delay_cause',
 'carrier_name',
 'airport_name',
 'Latitude',
 'Longitude',
 'delay_ct',
 'delay_min',
 'arr_category']

In [48]:
# reorder the columns 
df_delays = df_delays[['date', 'carrier', 'carrier_name', 'airport', 'airport_name', 'Latitude', 'Longitude',
                       'arr_category', 'delay_cause', 'delay_ct', 'delay_min']]

In [49]:
df_delays.sample(5)

,date,carrier,carrier_name,airport,airport_name,Latitude,Longitude,arr_category,delay_cause,delay_ct,delay_min
224216,2013-04-01,OO,SkyWest Airlines Inc.,PHX,"Phoenix, AZ: Phoenix Sky Harbor International",33.4342,112.0117,arr_del15,late_aircraft,58.15,3765.0
124528,2008-08-01,WN,Southwest Airlines Co.,MCO,"Orlando, FL: Orlando International",28.4294,81.3089,arr_del15,security,0.53,23.0
98300,2007-08-01,AA,American Airlines Inc.,IAD,"Washington, DC: Washington Dulles International",38.9475,77.4600,arr_del15,carrier,42.58,2424.0
294682,2017-04-01,AA,American Airlines Inc.,FLL,"Fort Lauderdale, FL: Fort Lauderdale-Hollywood...",26.0726,80.1528,arr_del15,nas,46.62,2163.0
310776,2018-03-01,B6,JetBlue Airways,BOS,"Boston, MA: Logan International",42.3631,71.0064,arr_del15,late_aircraft,525.56,34724.0


In [50]:
df_delays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318885 entries, 0 to 318884
Data columns (total 11 columns):
date            318885 non-null datetime64[ns]
carrier         318885 non-null object
carrier_name    318885 non-null object
airport         318885 non-null object
airport_name    318885 non-null object
Latitude        318885 non-null float64
Longitude       318885 non-null float64
arr_category    318885 non-null object
delay_cause     318885 non-null object
delay_ct        318885 non-null float64
delay_min       318885 non-null float64
dtypes: datetime64[ns](1), float64(4), object(6)
memory usage: 26.8+ MB


### Store

In [51]:
# store master DataFrame as well as flights and delays DataFrames in separate csv files
df.to_csv('master_flight_delays.csv', index=None) 
df_flights.to_csv('flights.csv', index=None)
df_delays.to_csv('delays.csv', index=None)